In [1]:
import classes.SAMMed2DClass as c
import utils.analysis as anUt
import utils.prompt as prUt
from utils.image import read_im_gt
from utils.interactivity import iterate_2d

import numpy as np

# Obtain model
device = 'cuda'
sammed2d_checkpoint_path = "/home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth"
inferer = c.SAMMed2DInferer(sammed2d_checkpoint_path, device)


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations


True
*******load /home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth


In [2]:
# Obtain image, gt
img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
img, gt = read_im_gt(img_path, gt_path, 3)


In [24]:
# Experiment: n randomly sampled points from foreground
seed = 11121
# n_clicks = 1
n_clicks = 5
point_prompt = prUt.get_pos_clicks2D_row_major(gt, n_clicks, seed = seed)
segmentation = inferer.predict(img, point_prompt)
anUt.compute_dice(segmentation, gt)

Performing inference on slices: 100%|██████████| 45/45 [00:01<00:00, 25.83it/s]


0.9143276197755548

In [59]:
# Experiment: n randomly sampled points from foreground
seed = 11121
# n_clicks = 1
n_clicks = 5
point_prompt = prUt.get_pos_clicks2D_row_major(gt, n_clicks, seed = seed)
segmentation, low_res_logits = inferer.predict(img, point_prompt, return_low_res_logits= True)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 161.90it/s]


0.9143276197755548

In [3]:
# Experiment: Point propagation
seed = 1
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.point_propagation(inferer, img, seed_point, slices_to_infer, seed, n_clicks)
print(anUt.compute_dice(gt,segmentation))


Propagating up: 100%|██████████| 22/22 [00:01<00:00, 21.50it/s]


0.7157095065514357


In [4]:
# Experiment: Point propagation
seed = 1
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, low_res_logits, prompt = prUt.point_propagation(inferer, img, seed_point, slices_to_infer, seed, n_clicks, return_low_res_logits = True)
print(anUt.compute_dice(gt,segmentation))


Propagating up: 100%|██████████| 22/22 [00:00<00:00, 115.38it/s]


0.7157095065514357


In [15]:
import utils.interactivity as inter
from importlib import reload
reload(inter)

# Iteratively improve previous segmentation (from point propagation)
seed = 1
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, low_res_logits, prompt = prUt.point_propagation(inferer, img, seed_point, slices_to_infer, seed, n_clicks, return_low_res_logits=True)


initial_prompt = prompt
condition = 'dof'
dof_bound = 90
perf_bound = 0.85
dice_scores, dof, segmentations, prompts = inter.iterate_2d(inferer, img, gt, segmentation, low_res_logits, initial_prompt, inferer.pass_prev_prompts,
                                                      scribble_length = 0.6, contour_distance = 3, disk_size_range= (0,3),
                                                      init_dof = 5, perf_bound = 0.85, dof_bound = dof_bound, seed = seed, detailed = True, )

Propagating up: 100%|██████████| 22/22 [00:00<00:00, 111.44it/s]


In [21]:
# Iteratively improve previous segmentation (from point propagation)
seed = 11121
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.point_propagation(inferer, img, seed_point, slices_to_infer, seed, n_clicks)

initial_prompt = prompt
condition = 'dof'
dof_bound = 90
dice_scores, dof, segmentations, prompts = iterate_2d(inferer, img, gt, segmentation, initial_prompt, inferer.pass_prev_prompts,
                                                      scribble_length = 0.6, contour_distance = 3, disk_size_range= (0,3),
                                                      init_dof = 5, dof_bound = dof_bound, seed = seed, detailed = True)

Propagating up: 100%|██████████| 22/22 [00:00<00:00, 115.02it/s]


0.8177772392806715


AttributeError: 'NoneType' object has no attribute 'keys'

In [11]:
# Experiment: Box propagation

seed_box = prUt.get_seed_box(gt)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.box_propagation(inferer, img, seed_box, slices_to_infer)

print(anUt.compute_dice(gt,segmentation))

Propagating up: 100%|██████████| 12/12 [00:00<00:00, 66.52it/s]


0.5264982538428203
